In [1]:
pip install torch torchvision transformers


In [2]:
import torch
from transformers import LxmertForPreTraining, LxmertTokenizer
from PIL import Image
import requests
from io import BytesIO
from torchvision import transforms

In [3]:
model = LxmertForPreTraining.from_pretrained("unc-nlp/lxmert-base-uncased")
tokenizer = LxmertTokenizer.from_pretrained("unc-nlp/lxmert-base-uncased")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.01G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/153 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [4]:
preprocess = transforms.Compose([
    transforms.Resize(800),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [11]:
import torch
from transformers import LxmertTokenizer, LxmertModel, LxmertProcessor
from PIL import Image
import torch.nn.functional as F
from torch import nn

ImportError: cannot import name 'LxmertProcessor' from 'transformers' (/usr/local/lib/python3.11/dist-packages/transformers/__init__.py)

In [10]:


class MemeAnalyzer:
    def __init__(self, device='cuda' if torch.cuda.is_available() else 'cpu'):
        self.device = device
        self.processor = LxmertProcessor.from_pretrained("unc-nlp/lxmert-base-uncased")
        self.model = LxmertModel.from_pretrained("unc-nlp/lxmert-base-uncased").to(device)
        self.caption_classifier = nn.Linear(768, 2).to(device)

        # VQA questions for comprehensive image understanding
        self.vqa_questions = [
            "What is happening in this image?",
            "What objects can you see?",
            "What are the people doing?",
            "What is the main subject?",
            "What is the setting or background?"
        ]

    def get_vqa_features(self, image, question):
        inputs = self.processor(
            images=image,
            text=question,
            return_tensors="pt",
            padding='max_length',
            max_length=128,
            truncation=True
        )
        return {k: v.to(self.device) for k, v in inputs.items()}

    def generate_caption(self, image):
        captions = []

        for question in self.vqa_questions:
            inputs = self.get_vqa_features(image, question)

            with torch.no_grad():
                outputs = self.model(**inputs)
                pooled_output = outputs.pooled_output

                # Use cross-attention weights to identify relevant image regions
                cross_attentions = outputs.cross_encoder_attentions
                attention_weights = torch.mean(cross_attentions[-1], dim=1)

                # Get most attended visual features
                visual_features = inputs['visual_feats']
                weighted_features = torch.matmul(attention_weights, visual_features)

                # Generate answer using the weighted features
                answer_logits = F.linear(weighted_features, self.model.pooler.dense.weight)
                answer_probs = F.softmax(answer_logits, dim=-1)

                # Convert to text (simplified for demonstration)
                response = self._convert_logits_to_text(answer_probs)
                captions.append(f"{question.strip('?')}: {response}")

        return " | ".join(captions)

    def _convert_logits_to_text(self, logits):
        # Placeholder for actual text generation
        # In practice, you'd use a vocabulary mapping
        return "Generated response based on attention weights"

    def analyze_meme(self, image_path):
        image = Image.open(image_path).convert('RGB')

        # Generate comprehensive caption using VQA
        caption = self.generate_caption(image)

        # Get features for hate detection
        inputs = self.get_vqa_features(image, caption)

        with torch.no_grad():
            outputs = self.model(**inputs)
            hate_predictions = self.caption_classifier(outputs.pooled_output)
            hate_probs = F.softmax(hate_predictions, dim=1)

        return {
            'caption': caption,
            'hateful': hate_probs[0][1].item(),
            'non_hateful': hate_probs[0][0].item()
        }

def main():
    analyzer = MemeAnalyzer()
    result = analyzer.analyze_meme('/content/me2.jpg')

    print(f"Image Analysis:\n{result['caption']}")
    print(f"Hateful Probability: {result['hateful']:.2%}")
    print(f"Non-hateful Probability: {result['non_hateful']:.2%}")

if __name__ == "__main__":
    main()

ImportError: cannot import name 'LxmertProcessor' from 'transformers' (/usr/local/lib/python3.11/dist-packages/transformers/__init__.py)

In [9]:
import torch
from transformers import LxmertTokenizer, LxmertModel, LxmertConfig
from PIL import Image
import torchvision.transforms as transforms
from torch import nn
import torch.nn.functional as F

class MemeClassifier(nn.Module):
    def __init__(self, num_labels=2):
        super().__init__()
        self.lxmert = LxmertModel.from_pretrained('unc-nlp/lxmert-base-uncased')
        self.classifier = nn.Linear(768, num_labels)

    def forward(self, input_ids, attention_mask, visual_feats, visual_pos):
        outputs = self.lxmert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            visual_feats=visual_feats,
            visual_pos=visual_pos
        )
        pooled_output = outputs.pooled_output
        return self.classifier(pooled_output)

class MemeAnalyzer:
    def __init__(self, device='cuda' if torch.cuda.is_available() else 'cpu'):
        self.device = device
        self.tokenizer = LxmertTokenizer.from_pretrained('unc-nlp/lxmert-base-uncased')
        self.model = MemeClassifier().to(device)
        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    def extract_visual_features(self, image):
        """Extract visual features using a pre-trained object detector"""
        # Placeholder for actual object detection
        # In practice, use Faster R-CNN or similar
        batch_size = 1
        num_objects = 36
        feat_dim = 2048
        pos_dim = 4

        # Dummy features for demonstration
        visual_feats = torch.randn(batch_size, num_objects, feat_dim).to(self.device)
        visual_pos = torch.randn(batch_size, num_objects, pos_dim).to(self.device)
        return visual_feats, visual_pos

    def analyze_meme(self, image_path, text):
        # Load and preprocess image
        image = Image.open(image_path).convert('RGB')
        image = self.transform(image).unsqueeze(0).to(self.device)

        # Extract visual features
        visual_feats, visual_pos = self.extract_visual_features(image)

        # Tokenize text
        inputs = self.tokenizer(
            text,
            padding='max_length',
            max_length=128,
            truncation=True,
            return_tensors="pt"
        )

        input_ids = inputs['input_ids'].to(self.device)
        attention_mask = inputs['attention_mask'].to(self.device)

        # Get prediction
        with torch.no_grad():
            outputs = self.model(input_ids, attention_mask, visual_feats, visual_pos)
            predictions = F.softmax(outputs, dim=1)

        return {
            'hateful': predictions[0][1].item(),
            'non_hateful': predictions[0][0].item()
        }

# Usage example
def main():
    analyzer = MemeAnalyzer()

    # Load model weights (assuming you have trained the model)
    # analyzer.model.load_state_dict(torch.load('path_to_model_weights.pth'))

    result = analyzer.analyze_meme(
        image_path='/content/me2.jpg',
        text='text extracted from meme'
    )

    print(f"Probability of being hateful: {result['hateful']:.2%}")
    print(f"Probability of being non-hateful: {result['non_hateful']:.2%}")

if __name__ == "__main__":
    main()

Some weights of the model checkpoint at unc-nlp/lxmert-base-uncased were not used when initializing LxmertModel: ['answer_head.logit_fc.0.bias', 'answer_head.logit_fc.0.weight', 'answer_head.logit_fc.2.bias', 'answer_head.logit_fc.2.weight', 'answer_head.logit_fc.3.bias', 'answer_head.logit_fc.3.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'obj_predict_head.decoder_dict.attr.bias', 'obj_predict_head.decoder_dict.attr.weight', 'obj_predict_head.decoder_dict.feat.bias', 'obj_predict_head.decoder_dict.feat.weight', 'obj_predict_head.decoder_dict.obj.bias', 'obj_predict_head.decoder_dict.obj.weight', 'obj_predict_head.transform.LayerNorm.bias', 'obj_predict_head.transform.LayerNorm.weight', 'obj_predict_head.transform.dense.bias', 'obj_pred

Probability of being hateful: 42.34%
Probability of being non-hateful: 57.66%


In [13]:
import torch
from transformers import LxmertTokenizer, LxmertModel, LxmertConfig
from PIL import Image
import torchvision.transforms as transforms
import torch.nn.functional as F
from torch import nn

In [14]:


class MemeAnalyzer:
    def __init__(self, device='cuda' if torch.cuda.is_available() else 'cpu'):
        self.device = device
        self.tokenizer = LxmertTokenizer.from_pretrained("unc-nlp/lxmert-base-uncased")
        self.model = LxmertModel.from_pretrained("unc-nlp/lxmert-base-uncased").to(device)
        self.caption_classifier = nn.Linear(768, 2).to(device)

        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

        self.vqa_questions = [
            "What is happening in this image?",
            "What objects can you see?",
            "What are the people doing?",
            "What is the main subject?",
            "What is the setting or background?"
        ]

    def get_vqa_features(self, image, question):
        # Process image
        image_tensor = self.transform(image).unsqueeze(0).to(self.device)

        # Process text
        encoding = self.tokenizer(
            question,
            padding='max_length',
            max_length=128,
            truncation=True,
            return_tensors="pt"
        )

        # Generate visual features (placeholder - replace with actual object detector)
        batch_size = 1
        num_objects = 36
        feat_dim = 2048
        pos_dim = 4
        visual_feats = torch.randn(batch_size, num_objects, feat_dim).to(self.device)
        visual_pos = torch.randn(batch_size, num_objects, pos_dim).to(self.device)

        return {
            'input_ids': encoding['input_ids'].to(self.device),
            'attention_mask': encoding['attention_mask'].to(self.device),
            'visual_feats': visual_feats,
            'visual_pos': visual_pos
        }

    def generate_caption(self, image):
        captions = []

        for question in self.vqa_questions:
            inputs = self.get_vqa_features(image, question)

            with torch.no_grad():
                outputs = self.model(**inputs)
                pooled_output = outputs.pooled_output

                # Use cross-attention weights
                cross_attentions = outputs.cross_encoder_attentions
                attention_weights = torch.mean(cross_attentions[-1], dim=1)

                # Get most attended features
                visual_features = inputs['visual_feats']
                weighted_features = torch.matmul(attention_weights, visual_features)

                # Generate answer
                answer_logits = self.caption_classifier(weighted_features)
                answer_probs = F.softmax(answer_logits, dim=-1)

                response = self._convert_logits_to_text(answer_probs)
                captions.append(f"{question.strip('?')}: {response}")

        return " | ".join(captions)

    def _convert_logits_to_text(self, logits):
        # Placeholder - implement with vocabulary mapping
        return "Generated response based on attention weights"

    def analyze_meme(self, image_path):
        image = Image.open(image_path).convert('RGB')
        caption = self.generate_caption(image)
        inputs = self.get_vqa_features(image, caption)

        with torch.no_grad():
            outputs = self.model(**inputs)
            hate_predictions = self.caption_classifier(outputs.pooled_output)
            hate_probs = F.softmax(hate_predictions, dim=1)

        return {
            'caption': caption,
            'hateful': hate_probs[0][1].item(),
            'non_hateful': hate_probs[0][0].item()
        }

# Required installations:
# !pip install torch torchvision transformers pillow

def main():
    analyzer = MemeAnalyzer()
    result = analyzer.analyze_meme('/content/me2.jpg')

    print(f"Image Analysis:\n{result['caption']}")
    print(f"Hateful Probability: {result['hateful']:.2%}")
    print(f"Non-hateful Probability: {result['non_hateful']:.2%}")

if __name__ == "__main__":
    main()

Some weights of the model checkpoint at unc-nlp/lxmert-base-uncased were not used when initializing LxmertModel: ['answer_head.logit_fc.0.bias', 'answer_head.logit_fc.0.weight', 'answer_head.logit_fc.2.bias', 'answer_head.logit_fc.2.weight', 'answer_head.logit_fc.3.bias', 'answer_head.logit_fc.3.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'obj_predict_head.decoder_dict.attr.bias', 'obj_predict_head.decoder_dict.attr.weight', 'obj_predict_head.decoder_dict.feat.bias', 'obj_predict_head.decoder_dict.feat.weight', 'obj_predict_head.decoder_dict.obj.bias', 'obj_predict_head.decoder_dict.obj.weight', 'obj_predict_head.transform.LayerNorm.bias', 'obj_predict_head.transform.LayerNorm.weight', 'obj_predict_head.transform.dense.bias', 'obj_pred

TypeError: 'NoneType' object is not subscriptable

In [16]:
class MemeAnalyzer:
    def __init__(self, device='cuda' if torch.cuda.is_available() else 'cpu'):
        self.device = device
        self.tokenizer = LxmertTokenizer.from_pretrained("unc-nlp/lxmert-base-uncased")
        self.model = LxmertModel.from_pretrained("unc-nlp/lxmert-base-uncased", output_attentions=True).to(device)
        self.caption_classifier = nn.Linear(768, 2).to(device)

        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

        self.vqa_questions = [
            "What is happening in this image?",
            "What objects can you see?",
            "What are the people doing?",
            "What is the main subject?",
            "What is the setting or background?"
        ]

    def get_vqa_features(self, image, question):
        image_tensor = self.transform(image).unsqueeze(0).to(self.device)

        encoding = self.tokenizer(
            question,
            padding='max_length',
            max_length=128,
            truncation=True,
            return_tensors="pt"
        )

        # Visual features
        batch_size = 1
        num_objects = 36
        feat_dim = 2048
        pos_dim = 4
        visual_feats = torch.randn(batch_size, num_objects, feat_dim).to(self.device)
        visual_pos = torch.randn(batch_size, num_objects, pos_dim).to(self.device)

        return {
            'input_ids': encoding['input_ids'].to(self.device),
            'attention_mask': encoding['attention_mask'].to(self.device),
            'visual_feats': visual_feats,
            'visual_pos': visual_pos
        }

    def generate_caption(self, image):
        captions = []

        for question in self.vqa_questions:
            inputs = self.get_vqa_features(image, question)

            with torch.no_grad():
                outputs = self.model(**inputs)
                pooled_output = outputs.pooled_output

                # Generate answer using pooled output directly
                answer_logits = self.caption_classifier(pooled_output)

                # Simple answer generation based on logits
                answer_probs = F.softmax(answer_logits, dim=-1)
                response = "Detected" if answer_probs[0][1] > 0.5 else "Not detected"

                captions.append(f"{question.strip('?')}: {response}")

        return " | ".join(captions)

    def analyze_meme(self, image_path):
        image = Image.open(image_path).convert('RGB')
        caption = self.generate_caption(image)
        inputs = self.get_vqa_features(image, caption)

        with torch.no_grad():
            outputs = self.model(**inputs)
            hate_predictions = self.caption_classifier(outputs.pooled_output)
            hate_probs = F.softmax(hate_predictions, dim=1)

        return {
            'caption': caption,
            'hateful': hate_probs[0][1].item(),
            'non_hateful': hate_probs[0][0].item()
        }

def main():
    analyzer = MemeAnalyzer()
    result = analyzer.analyze_meme('/content/me2.jpg')

    print(f"Image Analysis:\n{result['caption']}")
    print(f"Hateful Probability: {result['hateful']:.2%}")
    print(f"Non-hateful Probability: {result['non_hateful']:.2%}")

if __name__ == "__main__":
    main()

Some weights of the model checkpoint at unc-nlp/lxmert-base-uncased were not used when initializing LxmertModel: ['answer_head.logit_fc.0.bias', 'answer_head.logit_fc.0.weight', 'answer_head.logit_fc.2.bias', 'answer_head.logit_fc.2.weight', 'answer_head.logit_fc.3.bias', 'answer_head.logit_fc.3.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'obj_predict_head.decoder_dict.attr.bias', 'obj_predict_head.decoder_dict.attr.weight', 'obj_predict_head.decoder_dict.feat.bias', 'obj_predict_head.decoder_dict.feat.weight', 'obj_predict_head.decoder_dict.obj.bias', 'obj_predict_head.decoder_dict.obj.weight', 'obj_predict_head.transform.LayerNorm.bias', 'obj_predict_head.transform.LayerNorm.weight', 'obj_predict_head.transform.dense.bias', 'obj_pred

Image Analysis:
What is happening in this image: Detected | What objects can you see: Detected | What are the people doing: Detected | What is the main subject: Detected | What is the setting or background: Detected
Hateful Probability: 56.66%
Non-hateful Probability: 43.34%


In [17]:
import torch
from transformers import LxmertTokenizer, LxmertForQuestionAnswering
from PIL import Image
import torchvision.transforms as transforms
import torch.nn.functional as F
from torch import nn

In [19]:


class MemeAnalyzer:
    def __init__(self, device='cuda' if torch.cuda.is_available() else 'cpu'):
        self.device = device
        self.tokenizer = LxmertTokenizer.from_pretrained("unc-nlp/lxmert-base-uncased")
        self.model = LxmertForQuestionAnswering.from_pretrained("unc-nlp/lxmert-base-uncased").to(device)

        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

        self.vqa_questions = [
            "What is happening in this image?",
            "What objects can you see in the image?",
            "What are the people doing in this image?",
            "What is the main subject of this image?",
            "What is the setting or background of this image?"
        ]

    def get_vqa_features(self, image, question):
        image_tensor = self.transform(image).unsqueeze(0).to(self.device)

        encoding = self.tokenizer(
            question,
            padding='max_length',
            max_length=128,
            truncation=True,
            return_tensors="pt"
        )

        # Visual features (using random features for demonstration)
        batch_size = 1
        num_objects = 36
        feat_dim = 2048
        pos_dim = 4
        visual_feats = torch.randn(batch_size, num_objects, feat_dim).to(self.device)
        visual_pos = torch.randn(batch_size, num_objects, pos_dim).to(self.device)

        return {
            'input_ids': encoding['input_ids'].to(self.device),
            'attention_mask': encoding['attention_mask'].to(self.device),
            'visual_feats': visual_feats,
            'visual_pos': visual_pos
        }

    def answer_question(self, inputs):
        outputs = self.model(**inputs)
        answer_idx = outputs.question_answering_score.argmax(-1)
        return self.model.config.id2label[answer_idx.item()]

    def generate_caption(self, image):
        captions = []

        for question in self.vqa_questions:
            inputs = self.get_vqa_features(image, question)
            answer = self.answer_question(inputs)
            captions.append(f"{question.strip('?')}: {answer}")

        return " | ".join(captions)

    def analyze_meme(self, image_path):
        image = Image.open(image_path).convert('RGB')
        caption = self.generate_caption(image)

        # Use caption for hate detection
        inputs = self.get_vqa_features(image, caption)
        outputs = self.model(**inputs)
        hate_score = F.softmax(outputs.question_answering_score, dim=-1)

        return {
            'caption': caption,
            'hateful': hate_score[0][1].item(),
            'non_hateful': hate_score[0][0].item()
        }

def main():
    analyzer = MemeAnalyzer()
    result = analyzer.analyze_meme('/content/me2.jpg')

    print(f"Image Analysis:\n{result['caption']}")
    print(f"Hateful Probability: {result['hateful']:.2%}")
    print(f"Non-hateful Probability: {result['non_hateful']:.2%}")

if __name__ == "__main__":
    main()

KeyError: 4396

In [20]:
import torch
from transformers import LxmertTokenizer, LxmertForQuestionAnswering
from PIL import Image
import torchvision.transforms as transforms
import torch.nn.functional as F
import json
import requests

In [22]:


class MemeAnalyzer:
    def __init__(self, device='cuda' if torch.cuda.is_available() else 'cpu'):
        self.device = device
        self.tokenizer = LxmertTokenizer.from_pretrained("unc-nlp/lxmert-base-uncased")
        self.model = LxmertForQuestionAnswering.from_pretrained("unc-nlp/lxmert-base-uncased").to(device)

        # Load VQA vocabulary from Visual Genome dataset
        self.answer_vocab = self.load_vqa_vocabulary()

        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

        self.vqa_questions = [
            "What is happening in this image?",
            "What objects can you see in the image?",
            "What are the people doing in this image?",
            "What is the main subject of this image?",
            "What is the setting or background of this image?",
            "What emotions are expressed in this image?",
            "What text is visible in this image?",
            "What colors are prominent in this image?",
            "What is the relationship between elements in this image?",
            "What time of day/weather conditions are shown?"
        ]

    def load_vqa_vocabulary(self):
        # VQA 2.0 vocabulary URL
        vqa_vocab_url = "https://raw.githubusercontent.com/airsplay/lxmert/master/data/vqa/trainval_label2ans.json"

        try:
            response = requests.get(vqa_vocab_url)
            vocab = json.loads(response.text)
            return vocab
        except:
            # Fallback: Load from model config
            return list(self.model.config.label2ans.values())

    def get_vqa_features(self, image, question):
        image_tensor = self.transform(image).unsqueeze(0).to(self.device)

        encoding = self.tokenizer(
            question,
            padding='max_length',
            max_length=128,
            truncation=True,
            return_tensors="pt"
        )

        # Using features from a pre-trained object detector would be ideal here
        batch_size = 1
        num_objects = 36
        feat_dim = 2048
        pos_dim = 4
        visual_feats = torch.randn(batch_size, num_objects, feat_dim).to(self.device)
        visual_pos = torch.randn(batch_size, num_objects, pos_dim).to(self.device)

        return {
            'input_ids': encoding['input_ids'].to(self.device),
            'attention_mask': encoding['attention_mask'].to(self.device),
            'visual_feats': visual_feats,
            'visual_pos': visual_pos
        }

    def answer_question(self, inputs):
        outputs = self.model(**inputs)
        scores = F.softmax(outputs.question_answering_score, dim=-1)
        top_scores, top_indices = scores.topk(5)  # Get top 5 answers

        answers = []
        for score, idx in zip(top_scores[0], top_indices[0]):
            if idx < len(self.answer_vocab):
                answer = self.model.config.label2ans[idx]
                confidence = score.item()
                answers.append(f"{answer} ({confidence:.2%})")

        return ', '.join(answers) if answers else "Unable to detect"

    def generate_caption(self, image):
        captions = []

        for question in self.vqa_questions:
            inputs = self.get_vqa_features(image, question)
            answer = self.answer_question(inputs)
            captions.append(f"{question.strip('?')}: {answer}")

        return " | ".join(captions)

    def analyze_meme(self, image_path):
        image = Image.open(image_path).convert('RGB')
        caption = self.generate_caption(image)

        hate_inputs = self.get_vqa_features(image, "Is this image hateful or offensive?")
        hate_outputs = self.model(**hate_inputs)
        hate_score = F.softmax(hate_outputs.question_answering_score, dim=-1)

        return {
            'caption': caption,
            'analysis': self.analyze_content(caption),
            'hateful': hate_score[0][1].item(),
            'non_hateful': hate_score[0][0].item()
        }

    def analyze_content(self, caption):
        # Analyze the detected content for potential harmful elements
        analysis = []
        inputs = self.get_vqa_features(Image.new('RGB', (224, 224)), caption)
        outputs = self.model(**inputs)

        # Add content analysis based on the model's understanding
        context_score = F.softmax(outputs.question_answering_score, dim=-1)
        if context_score[0][1].item() > 0.5:
            analysis.append("Potentially sensitive content detected")

        return ' | '.join(analysis) if analysis else "No concerning content detected"

def main():
    analyzer = MemeAnalyzer()
    result = analyzer.analyze_meme('/content/meme4.jpg')

    print("Image Analysis:")
    print(result['caption'])
    print("\nContent Analysis:")
    print(result['analysis'])
    print(f"\nHateful Content Probability: {result['hateful']:.2%}")
    print(f"Non-hateful Content Probability: {result['non_hateful']:.2%}")

if __name__ == "__main__":
    main()

AttributeError: 'LxmertConfig' object has no attribute 'label2ans'

In [23]:
import torch
from transformers import LxmertTokenizer, LxmertForQuestionAnswering
from PIL import Image
import requests
from io import BytesIO
import json
import torch.nn.functional as F
from torchvision import transforms

In [24]:


class MemeAnalyzer:
    def __init__(self, device='cuda' if torch.cuda.is_available() else 'cpu'):
        self.device = device
        self.tokenizer = LxmertTokenizer.from_pretrained("unc-nlp/lxmert-base-uncased")
        self.model = LxmertForQuestionAnswering.from_pretrained("unc-nlp/lxmert-base-uncased").to(device)

        # Load VQA vocabulary from Visual Genome dataset
        self.answer_vocab = self.load_vqa_vocabulary()

        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

        self.vqa_questions = [
            "What is happening in this image?",
            "What objects can you see in this image?",
            "What are the people doing in this image?",
            "What is the main subject of this image?",
            "What is the setting or background of this image?",
            "What emotions are expressed in this image?",
            "What text is visible in this image?",
            "What colors are prominent in this image?",
            "What is the relationship between elements in this image?",
            "What time of day/weather conditions are shown?"
        ]

    def load_vqa_vocabulary(self):
        # VQA 2.0 vocabulary URL
        vqa_vocab_url = "https://raw.githubusercontent.com/airsplay/lxmert/master/data/vqa/trainval_label2ans.json"

        try:
            response = requests.get(vqa_vocab_url)
            vocab = json.loads(response.text)
            return vocab
        except:
            # Fallback: Use a predefined answer list
            return ["Yes", "No", "Maybe", "Unknown"]  # Example of possible answers

    def get_vqa_features(self, image, question):
        image_tensor = self.transform(image).unsqueeze(0).to(self.device)

        encoding = self.tokenizer(
            question,
            padding='max_length',
            max_length=128,
            truncation=True,
            return_tensors="pt"
        )

        # Using features from a pre-trained object detector would be ideal here
        batch_size = 1
        num_objects = 36
        feat_dim = 2048
        pos_dim = 4
        visual_feats = torch.randn(batch_size, num_objects, feat_dim).to(self.device)
        visual_pos = torch.randn(batch_size, num_objects, pos_dim).to(self.device)

        return {
            'input_ids': encoding['input_ids'].to(self.device),
            'attention_mask': encoding['attention_mask'].to(self.device),
            'visual_feats': visual_feats,
            'visual_pos': visual_pos
        }

    def answer_question(self, inputs):
        outputs = self.model(**inputs)
        scores = F.softmax(outputs.start_logits, dim=-1)  # Softmax on start logits
        end_scores = F.softmax(outputs.end_logits, dim=-1)  # Softmax on end logits

        # Get the top 5 answer spans
        start_scores, start_indices = scores.topk(5)
        end_scores, end_indices = end_scores.topk(5)

        answers = []
        for start_idx, end_idx, start_score, end_score in zip(start_indices[0], end_indices[0], start_scores[0], end_scores[0]):
            answer = self.tokenizer.decode(inputs['input_ids'][0][start_idx:end_idx + 1])
            confidence = (start_score.item() + end_score.item()) / 2  # Average of start and end scores
            answers.append(f"{answer} ({confidence:.2%})")

        return ', '.join(answers) if answers else "Unable to detect"

    def generate_caption(self, image):
        captions = []

        for question in self.vqa_questions:
            inputs = self.get_vqa_features(image, question)
            answer = self.answer_question(inputs)
            captions.append(f"{question.strip('?')}: {answer}")

        return " | ".join(captions)

    def analyze_meme(self, image_path):
        image = Image.open(image_path).convert('RGB')
        caption = self.generate_caption(image)

        hate_inputs = self.get_vqa_features(image, "Is this image hateful or offensive?")
        hate_outputs = self.model(**hate_inputs)
        hate_score = F.softmax(hate_outputs.start_logits, dim=-1)

        return {
            'caption': caption,
            'analysis': self.analyze_content(caption),
            'hateful': hate_score[0][1].item(),
            'non_hateful': hate_score[0][0].item()
        }

    def analyze_content(self, caption):
        # Analyze the detected content for potential harmful elements
        analysis = []
        inputs = self.get_vqa_features(Image.new('RGB', (224, 224)), caption)
        outputs = self.model(**inputs)

        # Add content analysis based on the model's understanding
        context_score = F.softmax(outputs.start_logits, dim=-1)
        if context_score[0][1].item() > 0.5:
            analysis.append("Potentially sensitive content detected")

        return ' | '.join(analysis) if analysis else "No concerning content detected"

def main():
    analyzer = MemeAnalyzer()
    result = analyzer.analyze_meme('/content/meme4.jpg')

    print("Image Analysis:")
    print(result['caption'])
    print("\nContent Analysis:")
    print(result['analysis'])
    print(f"\nHateful Content Probability: {result['hateful']:.2%}")
    print(f"Non-hateful Content Probability: {result['non_hateful']:.2%}")

if __name__ == "__main__":
    main()


AttributeError: 'LxmertForQuestionAnsweringOutput' object has no attribute 'start_logits'

In [25]:
import torch
from transformers import LxmertTokenizer, LxmertForQuestionAnswering
from PIL import Image
import requests
from io import BytesIO
import json
import torch.nn.functional as F
from torchvision import transforms

In [27]:


class MemeAnalyzer:
    def __init__(self, device='cuda' if torch.cuda.is_available() else 'cpu'):
        self.device = device
        self.tokenizer = LxmertTokenizer.from_pretrained("unc-nlp/lxmert-base-uncased")
        self.model = LxmertForQuestionAnswering.from_pretrained("unc-nlp/lxmert-base-uncased").to(device)

        # Load VQA vocabulary from Visual Genome dataset
        self.answer_vocab = self.load_vqa_vocabulary()

        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

        self.vqa_questions = [
            "What is happening in this image?",
            "What objects can you see in this image?",
            "What are the people doing in this image?",
            "What is the main subject of this image?",
            "What is the setting or background of this image?",
            "What emotions are expressed in this image?",
            "What text is visible in this image?",
            "What colors are prominent in this image?",
            "What is the relationship between elements in this image?",
            "What time of day/weather conditions are shown?"
        ]

    def load_vqa_vocabulary(self):
        # VQA 2.0 vocabulary URL
        vqa_vocab_url = "https://raw.githubusercontent.com/airsplay/lxmert/master/data/vqa/trainval_label2ans.json"

        try:
            response = requests.get(vqa_vocab_url)
            vocab = json.loads(response.text)
            return vocab
        except:
            # Fallback: Use a predefined answer list
            return ["Yes", "No", "Maybe", "Unknown"]  # Example of possible answers

    def get_vqa_features(self, image, question):
        image_tensor = self.transform(image).unsqueeze(0).to(self.device)

        encoding = self.tokenizer(
            question,
            padding='max_length',
            max_length=128,
            truncation=True,
            return_tensors="pt"
        )

        # Using features from a pre-trained object detector would be ideal here
        batch_size = 1
        num_objects = 36
        feat_dim = 2048
        pos_dim = 4
        visual_feats = torch.randn(batch_size, num_objects, feat_dim).to(self.device)
        visual_pos = torch.randn(batch_size, num_objects, pos_dim).to(self.device)

        return {
            'input_ids': encoding['input_ids'].to(self.device),
            'attention_mask': encoding['attention_mask'].to(self.device),
            'visual_feats': visual_feats,
            'visual_pos': visual_pos
        }

    def answer_question(self, inputs):
        outputs = self.model(**inputs)
        # Using question_answering_score to get the confidence of possible answers
        scores = F.softmax(outputs.question_answering_score, dim=-1)  # Softmax over all possible answers

        # Get top 5 answers
        top_scores, top_indices = scores.topk(5)

        answers = []
        for score, idx in zip(top_scores[0], top_indices[0]):
            if idx < len(self.answer_vocab):
                answer = self.answer_vocab[idx]  # Get answer from vocabulary
                confidence = score.item()
                answers.append(f"{answer} ({confidence:.2%})")

        return ', '.join(answers) if answers else "Unable to detect"

    def generate_caption(self, image):
        captions = []

        for question in self.vqa_questions:
            inputs = self.get_vqa_features(image, question)
            answer = self.answer_question(inputs)
            captions.append(f"{question.strip('?')}: {answer}")

        return " | ".join(captions)

    def analyze_meme(self, image_path):
        image = Image.open(image_path).convert('RGB')
        caption = self.generate_caption(image)

        hate_inputs = self.get_vqa_features(image, "Is this image hateful or offensive?")
        hate_outputs = self.model(**hate_inputs)
        hate_score = F.softmax(hate_outputs.question_answering_score, dim=-1)

        return {
            'caption': caption,
            'analysis': self.analyze_content(caption),
            'hateful': hate_score[0][1].item(),
            'non_hateful': hate_score[0][0].item()
        }

    def analyze_content(self, caption):
        # Analyze the detected content for potential harmful elements
        analysis = []
        inputs = self.get_vqa_features(Image.new('RGB', (224, 224)), caption)
        outputs = self.model(**inputs)

        # Add content analysis based on the model's understanding
        context_score = F.softmax(outputs.question_answering_score, dim=-1)
        if context_score[0][1].item() > 0.5:
            analysis.append("Potentially sensitive content detected")

        return ' | '.join(analysis) if analysis else "No concerning content detected"

def main():
    analyzer = MemeAnalyzer()
    result = analyzer.analyze_meme('/content/me2.jpg')

    print("Image Analysis:")
    print(result['caption'])
    print("\nContent Analysis:")
    print(result['analysis'])
    print(f"\nHateful Content Probability: {result['hateful']:.2%}")
    print(f"Non-hateful Content Probability: {result['non_hateful']:.2%}")

if __name__ == "__main__":
    main()


Image Analysis:
What is happening in this image: Unable to detect | What objects can you see in this image: pedestal (17.99%) | What are the people doing in this image: Unable to detect | What is the main subject of this image: beef (4.12%), lobster (2.95%) | What is the setting or background of this image: 11 (13.43%), not there (3.20%) | What emotions are expressed in this image: in corner (6.45%) | What text is visible in this image: woods (5.30%), chase (3.48%) | What colors are prominent in this image: throw frisbee (6.68%), riding horses (4.80%) | What is the relationship between elements in this image: honey (22.47%), bowl (14.71%), kid (2.31%) | What time of day/weather conditions are shown: 6:35 (2.61%), power lines (2.21%)

Content Analysis:
No concerning content detected

Hateful Content Probability: 0.01%
Non-hateful Content Probability: 0.00%


In [28]:
import torch
from transformers import LxmertTokenizer, LxmertForQuestionAnswering
from torchvision import models, transforms
import torch.nn.functional as F
from PIL import Image
import requests

class MemeAnalyzer:
    def __init__(self, device='cuda' if torch.cuda.is_available() else 'cpu'):
        self.device = device
        self.tokenizer = LxmertTokenizer.from_pretrained("unc-nlp/lxmert-base-uncased")
        self.model = LxmertForQuestionAnswering.from_pretrained("unc-nlp/lxmert-base-uncased").to(device)

        # Pretrained CNN for image feature extraction (e.g., ResNet)
        self.cnn_model = models.resnet50(pretrained=True).to(device)
        self.cnn_model.eval()  # Set the model to evaluation mode

        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

        # Predefined questions
        self.vqa_questions = [
            "What is happening in this image?",
            "What objects can you see in this image?",
            "What emotions are expressed in this image?",
            "What is the setting or background of this image?",
            "What is the main subject of this image?"
        ]

    def extract_image_features(self, image):
        # Process the image and extract features from a pretrained CNN (ResNet)
        image_tensor = self.transform(image).unsqueeze(0).to(self.device)
        with torch.no_grad():
            features = self.cnn_model(image_tensor)
        return features

    def get_vqa_features(self, image, question):
        image_features = self.extract_image_features(image)

        encoding = self.tokenizer(
            question,
            padding='max_length',
            max_length=128,
            truncation=True,
            return_tensors="pt"
        )

        # Flatten image features and position to match LXMERT input format
        visual_feats = image_features.unsqueeze(1)
        visual_pos = torch.zeros(visual_feats.shape).to(self.device)  # Placeholder for positions

        return {
            'input_ids': encoding['input_ids'].to(self.device),
            'attention_mask': encoding['attention_mask'].to(self.device),
            'visual_feats': visual_feats,
            'visual_pos': visual_pos
        }

    def answer_question(self, inputs):
        outputs = self.model(**inputs)
        start_logits = outputs.start_logits
        end_logits = outputs.end_logits
        start_scores = F.softmax(start_logits, dim=-1)  # Softmax on start logits
        end_scores = F.softmax(end_logits, dim=-1)  # Softmax on end logits

        return start_scores, end_scores

    def generate_caption(self, image):
        captions = []

        for question in self.vqa_questions:
            inputs = self.get_vqa_features(image, question)
            start_scores, end_scores = self.answer_question(inputs)

            # Select the most likely words based on the highest start and end scores
            start_index = torch.argmax(start_scores)
            end_index = torch.argmax(end_scores)
            caption = f"{question}: {start_index.item()}-{end_index.item()}"
            captions.append(caption)

        return " | ".join(captions)

    def analyze_meme(self, image_path):
        image = Image.open(image_path).convert('RGB')
        caption = self.generate_caption(image)

        hate_inputs = self.get_vqa_features(image, "Is this image hateful or offensive?")
        start_scores, end_scores = self.answer_question(hate_inputs)
        hate_score = F.softmax(start_scores, dim=-1)[0][1].item()  # Get hate score for "offensive"

        return {
            'caption': caption,
            'hate_score': hate_score
        }

def main():
    analyzer = MemeAnalyzer()
    result = analyzer.analyze_meme('/content/me2.jpg')  # Update with actual image path

    print("Image Analysis:")
    print(result['caption'])
    print(f"Hate Speech Probability: {result['hate_score']:.2%}")

if __name__ == "__main__":
    main()


RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x1000 and 2048x768)

In [29]:
import torch
from transformers import LxmertTokenizer, LxmertForQuestionAnswering
from torchvision import models, transforms
import torch.nn.functional as F
from PIL import Image
import requests

class MemeAnalyzer:
    def __init__(self, device='cuda' if torch.cuda.is_available() else 'cpu'):
        self.device = device
        self.tokenizer = LxmertTokenizer.from_pretrained("unc-nlp/lxmert-base-uncased")
        self.model = LxmertForQuestionAnswering.from_pretrained("unc-nlp/lxmert-base-uncased").to(device)

        # Pretrained CNN for image feature extraction (e.g., ResNet)
        self.cnn_model = models.resnet50(pretrained=True).to(device)
        self.cnn_model.eval()  # Set the model to evaluation mode

        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

        # Predefined questions
        self.vqa_questions = [
            "What is happening in this image?",
            "What objects can you see in this image?",
            "What emotions are expressed in this image?",
            "What is the setting or background of this image?",
            "What is the main subject of this image?"
        ]

    def extract_image_features(self, image):
        # Process the image and extract features from a pretrained CNN (ResNet)
        image_tensor = self.transform(image).unsqueeze(0).to(self.device)
        with torch.no_grad():
            features = self.cnn_model(image_tensor)

        # Reshape the features to match LXMERT's expected input shape (36 objects, each with 2048 dimensions)
        # We need to reshape the 1000-dimensional vector from ResNet to 36x2048 shape
        features = features.view(1, 36, 2048)  # This is a trick to reshape, but it needs careful attention
        return features

    def get_vqa_features(self, image, question):
        image_features = self.extract_image_features(image)

        encoding = self.tokenizer(
            question,
            padding='max_length',
            max_length=128,
            truncation=True,
            return_tensors="pt"
        )

        # Visual positions for LXMERT (using dummy values)
        visual_pos = torch.zeros(image_features.shape).to(self.device)  # Placeholder for positions

        return {
            'input_ids': encoding['input_ids'].to(self.device),
            'attention_mask': encoding['attention_mask'].to(self.device),
            'visual_feats': image_features,
            'visual_pos': visual_pos
        }

    def answer_question(self, inputs):
        outputs = self.model(**inputs)
        start_logits = outputs.start_logits
        end_logits = outputs.end_logits
        start_scores = F.softmax(start_logits, dim=-1)  # Softmax on start logits
        end_scores = F.softmax(end_logits, dim=-1)  # Softmax on end logits

        return start_scores, end_scores

    def generate_caption(self, image):
        captions = []

        for question in self.vqa_questions:
            inputs = self.get_vqa_features(image, question)
            start_scores, end_scores = self.answer_question(inputs)

            # Select the most likely words based on the highest start and end scores
            start_index = torch.argmax(start_scores)
            end_index = torch.argmax(end_scores)
            caption = f"{question}: {start_index.item()}-{end_index.item()}"
            captions.append(caption)

        return " | ".join(captions)

    def analyze_meme(self, image_path):
        image = Image.open(image_path).convert('RGB')
        caption = self.generate_caption(image)

        hate_inputs = self.get_vqa_features(image, "Is this image hateful or offensive?")
        start_scores, end_scores = self.answer_question(hate_inputs)
        hate_score = F.softmax(start_scores, dim=-1)[0][1].item()  # Get hate score for "offensive"

        return {
            'caption': caption,
            'hate_score': hate_score
        }

def main():
    analyzer = MemeAnalyzer()
    result = analyzer.analyze_meme('/content/me2.jpg')  # Update with actual image path

    print("Image Analysis:")
    print(result['caption'])
    print(f"Hate Speech Probability: {result['hate_score']:.2%}")

if __name__ == "__main__":
    main()


RuntimeError: shape '[1, 36, 2048]' is invalid for input of size 1000